In [ ]:
from stock_env.algos.agent import Agent
from stock_env.envs import *
from stock_env.common.common_utils import open_config
from stock_env.common.env_utils import make_vec_env
import torch as th
from stock_env.common.common_utils import create_performance, plot_trade_log_v2
from stock_env.common.evaluation import play_an_episode, evaluate_agent
from stock_env.common.common_utils import open_config
import seaborn as sns
sns.set()
import plotly.graph_objects as go
import os
from empyrical import (
    sharpe_ratio,
    max_drawdown,
)

In [ ]:
_ticker = ""
# VN30 = "ACB BCM BID BVH CTG FPT GAS GVR HDB HPG MBB MSN MWG NVL PDR PLX POW SAB SSI STB TCB TPB VCB VHM VIB VIC VJC VNM VPB VRE".split()
ppo_model_folder = f"../model/evaluation{_ticker}/"
ppo_adapt_model_folder = f"../model/adapt_evaluation{_ticker}/"
ppo_config = "../configs/ppo.yaml"

env_id = "VNALL-v0"
args = open_config(ppo_config, env_id=env_id)
envs = make_vec_env(env_id, num_envs=1, task=args.task)
agent = Agent(envs, hiddens=args.hiddens)
tickers = envs.envs[0].data_loader.tickers
# tickers = VN30
# tickers = [t for t in tickers if t.startswith(_ticker[1])]

# PPO trading performance 

In [ ]:
ppo_files = os.listdir(ppo_model_folder)
ppo_adapt_files = os.listdir(ppo_adapt_model_folder)

ppo_models, ppo_adapt_models = [], []
for filename in ppo_files:
    if filename.startswith("ppo_"):
        ppo_models.append(filename)

for filename in ppo_adapt_files:
    if filename.startswith("ppo_adapt_"):
        ppo_adapt_models.append(filename)

In [ ]:
# setting up
N = 10
multi_index = pd.MultiIndex.from_product([tickers, ["PPO", "PPO_adapt", "Buy-n-Hold"], [i for i in range(N)]], names=["Ticker", "Algorithm", "Ith"])
eval_df = pd.DataFrame(index=multi_index, columns=["Sharpe", "MaxDD"])
for file in ppo_models:
    task = file.split("_")[1]
    if task in tickers:
        model_path = os.path.join(ppo_model_folder, file)

        # prepare env and agent
        envs.reset_task(task)
        try:
            agent.load_state_dict(th.load(model_path))
        except:
            continue

        # evaluate
        envs.train(False)
        for i in range(N):
            info = play_an_episode(agent, envs)
            df = info["final_info"][0]["final_history"]
            returns = df.set_index("time")["portfolio_value"].pct_change()
            bnh_returns = df.set_index("time")["close_price"].pct_change()

            # metrics
            eval_df.loc[(task, 'PPO', i), :] = [sharpe_ratio(returns), -max_drawdown(returns) * 100]
            eval_df.loc[(task, 'Buy-n-Hold', i), :] = [sharpe_ratio(bnh_returns), -max_drawdown(bnh_returns) * 100]

for file in ppo_adapt_models:
    task = file.split("_")[2]
    if task in tickers:
        model_path = os.path.join(ppo_adapt_model_folder, file)

        # prepare env and agent
        envs.reset_task(task)
        try:
            agent.load_state_dict(th.load(model_path))
        except:
            continue

        # evaluate
        envs.train(False)
        for i in range(N):
            info = play_an_episode(agent, envs)
            df = info["final_info"][0]["final_history"]
            returns = df.set_index("time")["portfolio_value"].pct_change()

            # metrics
            eval_df.loc[(task, 'PPO_adapt', i), :] = [sharpe_ratio(returns), -max_drawdown(returns) * 100]

In [25]:
eval_df = eval_df.dropna()
eval_df = eval_df.reset_index()

In [26]:
eval_df.groupby('Algorithm').agg(['mean', 'std'])

level_0                     index                    Ith  \
                   mean          std         mean          std      mean   
Algorithm                                                                  
Buy-n-Hold  4639.632640  2670.867467  4639.632640  2670.867467  4.497480   
PPO         4620.537807  2670.986407  4620.537807  2670.986407  4.497480   
PPO_adapt   4597.870377  2661.650730  4597.870377  2661.650730  4.488075   

                        Sharpe                MaxDD             
                 std      mean       std       mean        std  
Algorithm                                                       
Buy-n-Hold  2.872623  0.569142  1.884011  33.231198  19.751082  
PPO         2.872623  0.439935  1.881626  19.452504  13.590096  
PPO_adapt   2.874438  0.347248  1.752136   9.990527   8.765177

In [ ]:
eval_df.to_csv(f"../temp/VNALL_eval_df.csv", index=False)

In [ ]:
# adjust figure size
import matplotlib.pyplot as plt
fig, ax = plt.subplots(2, 1, figsize=(30, 15))
sns.barplot(
    x="Ticker", 
    y="Sharpe", 
    hue="Algorithm",
    # ci="sd",
    data=eval_df.reset_index().sort_values("Sharpe", ascending=False),
    ax=ax[0],
)
sns.barplot(
    x="Ticker", 
    y="MaxDD", 
    hue="Algorithm",
    # ci="sd",
    data=eval_df.reset_index().sort_values("MaxDD", ascending=False),
    ax=ax[1]
)
fig.show()